In [ ]:
import plotly.express as px
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
from src.core.domain.use_cases.import_network_from_json import ETLPipeline
from src.core.infrastructure.settings import Settings
from src.core.infrastructure.adapters.sqlite_network_repository import (
    SQLiteNetworkRepository,
)
from src.core.infrastructure.adapters.network_builder import DefaultNetworkBuilder
from src.core.domain.use_cases.compute_simulated_network import SimulationPipeline
from src.core.infrastructure.services import PyPowsyblCompatService
from src.core.infrastructure.adapters.pypowsybl_loadflow_solver import PyPowSyblLoadFlowSolver
from src.core.constants import LoadFlowType
from src.core.utils import parse_datetime_to_str
from src.core.constants import DEFAULT_TIMEZONE
from src.rl.artifacts.experiment_record import ExperimentRecord
import src.rl.action as action_module
from src.rl.action.enums import DiscreteActionTypes
from src.core.constants import ElementStatus

PATH_TO_LAYOUT = "configs/toy_grid_layout.json"
SIMULATION_CONFIG_PATH = Path("configs/toy_grid_simulation.yaml")
GRID_ID = "toy_grid_layout"
SHOULD_CREATE_TABLE = True
settings = Settings()
network_builder = DefaultNetworkBuilder()

In [ ]:
file_path = Path(PATH_TO_LAYOUT)

In [ ]:
# Ingest the data

network_repository = SQLiteNetworkRepository(
    should_create_tables=SHOULD_CREATE_TABLE,
    db_url=settings.DB_URL,
)

etl_pipeline = ETLPipeline(
    network_repository=network_repository,
    network_builder=network_builder,
)
etl_pipeline.run(file_path=PATH_TO_LAYOUT)

In [ ]:
# Visualise the ingested data

net = network_repository.get(network_id=GRID_ID)

In [ ]:
net.elements

In [ ]:
# Simulate a time series of the network

START = datetime(2025, 1, 1, tzinfo=DEFAULT_TIMEZONE)
END = datetime(2025, 2, 1, tzinfo=DEFAULT_TIMEZONE)

simulation_pipeline = SimulationPipeline(
    config_path=SIMULATION_CONFIG_PATH,
    network_repository=network_repository,
    network_builder=network_builder,
)

simulation_pipeline.apply_pipeline(start=parse_datetime_to_str(START), end=parse_datetime_to_str(END), time_step=1)

In [ ]:
net_simulated = network_repository.get(network_id=f"{GRID_ID}_simulated")

In [ ]:
df = pd.concat(
    [
        net_simulated.to_dataframe(element_id="gen1")[["dynamic.Ptarget"]],
        net_simulated.to_dataframe(element_id="gen1")[["static.Pmax"]],
        net_simulated.to_dataframe(element_id="load1")[["dynamic.Pd"]],
    ],
    axis=1,
)
px.line(df)

In [ ]:
# Solve the network

solver = PyPowSyblLoadFlowSolver(
    to_pypowsybl_converter_service=PyPowsyblCompatService(),
    network_builder=network_builder,
)

net_solved = solver.solve(network=net_simulated, loadflow_type=LoadFlowType.DC)

In [ ]:
ELEMENT_ID_1 = "gen1"
ELEMENT_ID_2 = "load1"
line_1_id = "line1"

df_solved= pd.concat(
    [
        net_solved.to_dataframe(element_id=ELEMENT_ID_1)[["solved.p"]].rename(columns={"solved.p": f"solved.p_{ELEMENT_ID_1}"}),
        net_solved.to_dataframe(element_id="load1")[["solved.p"]].rename(columns={"solved.p": f"solved.p_{ELEMENT_ID_2}"}),
        net_solved.to_dataframe(element_id=line_1_id)[["solved.p1"]]
    ],
    axis=1,
)
px.line(df_solved)

In [ ]:
# Visualise an RL experiment layout

ROLLOUT_PATH = "src/rl/data/experiments/DQN-agent-exp/4716cdfe142645d99ade9b0207670e6f/artifacts/DQN-agent-exp_rollout_episode_250.json"

with open(Path(ROLLOUT_PATH), "r") as f:
    rl_experiment_layout = json.load(f)

In [ ]:
records = [ExperimentRecord(**i) for i in rl_experiment_layout["records"]]

In [ ]:
px.line(pd.DataFrame([j
 for record in records
 for j in record.next_observation["network_snapshot_observations"][-1]["observations"]
 if j["id"] == "line2"])[["timestamp", "p1"]].set_index("timestamp"))

In [ ]:
def assign_status(status: ElementStatus) -> int:
    if status == ElementStatus.ON:
        return 0
    elif status == ElementStatus.OFF:
        return 1
    elif status == ElementStatus.OUTAGE:
        return 2
    elif status == ElementStatus.MAINTENANCE:
        return 3

df_status = pd.DataFrame([j
 for record in records
 for j in record.next_observation["network_snapshot_observations"][-1]["observations"]
 if j["id"] == "line1"])[["timestamp", "status"]].set_index("timestamp")

df_status["status_code"] = df_status["status"].apply(assign_status)

px.line(df_status[["status_code"]])

In [ ]:
px.line(pd.DataFrame([i.reward for i in records]))

In [ ]:
def assign_action(action: dict, action_type: DiscreteActionTypes) -> int:
    try:
        action_class = getattr(action_module, action_type)
        if action_type == "DoNothingAction":
            action = {}
        if action_type == "SwitchAction":
            action = {"element_id": action["element_id"]}
        if action_type == "StartMaintenanceAction":
            action = {"element_id": action["element_id"]}
        action = action_class(**action)
    except ValueError as e:
        raise ValueError(f"Unknown action type: {action_type}: {e}")
    if isinstance(action, action_module.DoNothingAction):
        return 0
    elif isinstance(action, action_module.SwitchAction) and action.element_id == "line1":
        return 1
    elif isinstance(action, action_module.SwitchAction) and action.element_id == "line2":
        return 2
    elif isinstance(action, action_module.StartMaintenanceAction) and action.element_id == "line1":
        return 3
    elif isinstance(action, action_module.StartMaintenanceAction) and action.element_id == "line2":
        return 4

In [ ]:
px.line(pd.DataFrame([assign_action(i.action, action_type=i.action["action_type"]) for i in records]))